In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
import keras
from keras.datasets import mnist
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Dropout,Flatten
from keras.layers.advanced_activations import LeakyReLU

from keras.optimizers import Adam
from tqdm import tqdm

In [23]:
# reload the load_data
def load_data():
    (train_data,train_labels),(test_data,test_labels) = mnist.load_data()
    train_data = (train_data.astype(np.float32)-127.5)/127.5

    train_data = train_data.reshape(train_data.shape[0],-1)

    return (train_data,train_labels,test_data,test_labels)

In [24]:
train_data,train_labels,test_data,test_labels = load_data()

In [26]:
print("train_data.shape:{}".format(train_data.shape))
print("train_labels.shape:{}".format(train_labels.shape))
print("test_data.shape:{}".format(test_data.shape))
print("test_labels.shape:{}".format(test_labels.shape))

train_data.shape:(60000, 784)
train_labels.shape:(60000,)
test_data.shape:(10000, 28, 28)
test_labels.shape:(10000,)


In [27]:
# reload the optimizer
def adam_optimizer():
    return Adam(lr=0.0002,
                beta_1=0.5)

In [31]:
input_dim = 100

def create_generator(input_dim):
    generator = Sequential()
    generator.add(Dense(units=256,
                        input_dim=input_dim))
    generator.add(LeakyReLU(alpha=0.2))

    generator.add(Dense(units=512))
    generator.add(LeakyReLU(alpha=0.2))

    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(alpha=0.2))

    generator.add(Dense(units=784,
                  activation="tanh"))

    loss = "binary_crossentropy"
    optimizer = adam_optimizer()

    generator.compile(loss=loss,
                      optimizer=optimizer)
    
    return generator

In [32]:
generator = create_generator(input_dim)

In [33]:
generator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 784)              

In [37]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units=1024,
                       input_dim=784))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(rate=0.3))

    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(rate=0.3))

    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(alpha=0.2))

    discriminator.add(Dense(units=1,
                            activation="sigmoid"))

    loss = "binary_crossentropy"
    optimizer = adam_optimizer()

    discriminator.compile(loss=loss,
                          optimizer=optimizer)

    return discriminator

In [38]:
discriminator = create_discriminator()

In [39]:
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)              

In [49]:
def create_gan(generator,discriminator):
    discriminator.trainable = False
    gan_input = Input((100,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan_model = Model(gan_input,gan_output)
    loss = "binary_crossentropy"
    optimizer = "adam"

    gan_model.compile(loss=loss,
                      optimizer=optimizer)

    return gan_model

In [50]:
gan = create_gan(generator,discriminator)

In [51]:
gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [1]:
def plot_generated_images(epoch,generator,examples=100,dim=(10,10),figsize=(10,10)):
    noise = np.random.normal(loc=0.0,scale=1.0,size=(examples,100))
    generator_images = generator.predict(noise).reshape(examples,28,28)
    plt.figure(figsize=figsize)
    for i in range(generator_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        plt.imshow(generator_images[i],interpolation="nearest")
        plt.axis("off")

    plt.tight_layout()
    plt.savefig("gan_generator_image %d.png"%epoch)

In [ ]:
def training(epochs=1,batch_size=128):
    #load the data
    train_data,train_labels,test_data,test_labels = load_data()
    batch_count = train_data/batch_size

    # create gan-networks
    generator = create_generator(100)
    discriminator = create_discriminator()
    gan = create_gan(generator,discriminator)

    for epoch in range(1,epochs+1):
        print("Epoch %d " % epoch)
        for _ in tqdm()